### Link To CDC for Raw Data
### url: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

In [30]:
# import dependencies
import pandas as pd
import datetime as dt
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, inspect, func
import psycopg2
import requests
from sodapy import Socrata

# import cdc key for API
from cdc_token import my_token

## USA Monthly Cases by State 

In [31]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9mfq-cb36", limit=50000)

# Convert to pandas DataFrame and display
covid_df = pd.DataFrame.from_records(results)
covid_df.tail()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
26215,2020-12-19T00:00:00.000,RMI,4,4.0,0,0.0,0,0,0.0,0,0.0,0,2020-12-20T14:32:33.200,Agree,Agree
26216,2020-04-09T00:00:00.000,UT,2061,NaN,NaN,145.0,NaN,15,NaN,NaN,2.0,NaN,2020-04-08T16:22:39.452,Agree,Agree
26217,2020-10-19T00:00:00.000,KS,72968,67935.0,5033,2113.0,179,872,NaN,NaN,13.0,0,2020-10-20T14:59:27.967,Agree,N/A
26218,2020-12-05T00:00:00.000,NY,368246,NaN,NaN,7532.0,0,10274,NaN,NaN,57.0,0,2020-12-06T14:51:16.617,Not agree,Not agree
26219,2020-03-31T00:00:00.000,IA,497,NaN,NaN,73.0,NaN,7,NaN,NaN,1.0,NaN,2020-03-30T16:22:39.452,Not agree,Not agree


In [32]:
# add date columns which only has year and month for use with groupby later on.. 
covid_df['date'] = pd.to_datetime(covid_df['submission_date']).dt.to_period('M').astype(str)
covid_df.tail()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,date
26215,2020-12-19T00:00:00.000,RMI,4,4.0,0,0.0,0,0,0.0,0,0.0,0,2020-12-20T14:32:33.200,Agree,Agree,2020-12
26216,2020-04-09T00:00:00.000,UT,2061,NaN,NaN,145.0,NaN,15,NaN,NaN,2.0,NaN,2020-04-08T16:22:39.452,Agree,Agree,2020-04
26217,2020-10-19T00:00:00.000,KS,72968,67935.0,5033,2113.0,179,872,NaN,NaN,13.0,0,2020-10-20T14:59:27.967,Agree,N/A,2020-10
26218,2020-12-05T00:00:00.000,NY,368246,NaN,NaN,7532.0,0,10274,NaN,NaN,57.0,0,2020-12-06T14:51:16.617,Not agree,Not agree,2020-12
26219,2020-03-31T00:00:00.000,IA,497,NaN,NaN,73.0,NaN,7,NaN,NaN,1.0,NaN,2020-03-30T16:22:39.452,Not agree,Not agree,2020-03


In [33]:
covid_df.dtypes

submission_date    object
state              object
tot_cases          object
conf_cases         object
prob_cases         object
new_case           object
pnew_case          object
tot_death          object
conf_death         object
prob_death         object
new_death          object
pnew_death         object
created_at         object
consent_cases      object
consent_deaths     object
date               object
dtype: object

In [34]:
# convert objects to float
covid_df["tot_cases"] =  covid_df.tot_cases.astype(float)
covid_df["tot_death"] = covid_df.tot_death.astype(float)

# select only the columns we gonna use and display df
covid_df = covid_df[["date", "state", "tot_cases", "tot_death"]]
covid_df.tail()

,date,state,tot_cases,tot_death
26215,2020-12,RMI,4.0,0.0
26216,2020-04,UT,2061.0,15.0
26217,2020-10,KS,72968.0,872.0
26218,2020-12,NY,368246.0,10274.0
26219,2020-03,IA,497.0,7.0


In [86]:
# group by states to be used later
state_df = covid_df.groupby(['date', 'state']).sum().reset_index()
state_df

,date,state,tot_cases,tot_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,6.0,0.0,6.0
...,...,...,...,...,...
955,2021-04,VT,39278.0,455.0,38823.0
956,2021-04,WA,732877.0,10540.0,722337.0
957,2021-04,WI,1274709.0,14662.0,1260047.0
958,2021-04,WV,284886.0,5371.0,279515.0


In [87]:
# group by states to be used later
state_df = covid_df.groupby('state').sum().reset_index()
state_df

,state,tot_cases,tot_death,Total_recovered
0,AK,7685258.0,39256.0,7646002.0
1,AL,75341361.0,1308745.0,74032616.0
2,AR,46173819.0,745332.0,45428487.0
3,AS,452.0,0.0,452.0
4,AZ,117187972.0,2352839.0,114835133.0
5,CA,471170482.0,7196424.0,463974058.0
6,CO,61107651.0,1065917.0,60041734.0
7,CT,41392844.0,1788435.0,39604409.0
8,DC,7174618.0,234363.0,6940255.0
9,DE,12638669.0,263031.0,12375638.0


In [83]:
# group by states to be used later
state_df = covid_df.groupby('state').mean().reset_index()
state_df

,state,tot_cases,tot_death,Total_recovered
0,AK,1.758640e+04,89.830664,1.749657e+04
1,AL,1.724059e+05,2994.839817,1.694110e+05
2,AR,1.056609e+05,1705.565217,1.039553e+05
3,AS,1.034325e+00,0.000000,1.034325e+00
4,AZ,2.681647e+05,5384.070938,2.627806e+05
5,CA,1.078193e+06,16467.789474,1.061726e+06
6,CO,1.398344e+05,2439.169336,1.373953e+05
7,CT,9.472047e+04,4092.528604,9.062794e+04
8,DC,1.641789e+04,536.299771,1.588159e+04
9,DE,2.892144e+04,601.901602,2.831954e+04


In [36]:
# add total recovery columns to df
covid_df["Total_recovered"] = covid_df.tot_cases - covid_df.tot_death
covid_df.tail()

,date,state,tot_cases,tot_death,Total_recovered
26215,2020-12,RMI,4.0,0.0,4.0
26216,2020-04,UT,2061.0,15.0,2046.0
26217,2020-10,KS,72968.0,872.0,72096.0
26218,2020-12,NY,368246.0,10274.0,357972.0
26219,2020-03,IA,497.0,7.0,490.0


In [85]:
# groupby date and state and take the mean value to get monthly average..
covid_monthly_df = round(covid_df.groupby(["date", "state"]).mean().reset_index(), 2)
covid_monthly_df

,date,state,tot_cases,tot_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,0.6,0.0,0.6
...,...,...,...,...,...
955,2021-04,VT,19639.0,227.5,19411.5
956,2021-04,WA,366438.5,5270.0,361168.5
957,2021-04,WI,637354.5,7331.0,630023.5
958,2021-04,WV,142443.0,2685.5,139757.5


In [38]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_monthly_df = covid_monthly_df.rename(columns={"date": "Date", "state": "State", "tot_cases": "Average_cases", "tot_death": "Average_death", "Total_recovered": "Average_recovery"})
covid_monthly_df

,Date,State,Average_cases,Average_death,Average_recovery
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,0.6,0.0,0.6
...,...,...,...,...,...
955,2021-04,VT,19639.0,227.5,19411.5
956,2021-04,WA,366438.5,5270.0,361168.5
957,2021-04,WI,637354.5,7331.0,630023.5
958,2021-04,WV,142443.0,2685.5,139757.5


In [39]:
# add death and recovery percentage columns. change NA values to zero
covid_monthly_df['Death_percent'] = round((covid_monthly_df.Average_death/covid_monthly_df.Average_cases)*100, 2)
covid_monthly_df['Recovery_percent'] = round((covid_monthly_df.Average_recovery/covid_monthly_df.Average_cases)*100, 2)
covid_monthly_df=covid_monthly_df.fillna(0)
covid_monthly_df

,Date,State,Average_cases,Average_death,Average_recovery,Death_percent,Recovery_percent
0,2020-01,AK,0.0,0.0,0.0,0.00,0.00
1,2020-01,AL,0.0,0.0,0.0,0.00,0.00
2,2020-01,AR,0.0,0.0,0.0,0.00,0.00
3,2020-01,AS,0.0,0.0,0.0,0.00,0.00
4,2020-01,AZ,0.6,0.0,0.6,0.00,100.00
...,...,...,...,...,...,...,...
955,2021-04,VT,19639.0,227.5,19411.5,1.16,98.84
956,2021-04,WA,366438.5,5270.0,361168.5,1.44,98.56
957,2021-04,WI,637354.5,7331.0,630023.5,1.15,98.85
958,2021-04,WV,142443.0,2685.5,139757.5,1.89,98.11


In [40]:
# save df as a csv file
covid_monthly_df.to_csv('../Data/monthly.csv')

## USA Total Cases and Death by State

In [41]:
# upload data file and read it ito a pandas dataframe
file = "../Data/US_COVID-19_Deaths.csv"
covid_stats_df = pd.read_csv(file)
covid_stats_df.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,Death Rate per 100000,Case Rate per 100000 in Last 7 Days,Death Rate per 100K in Last 7 Days
0,Alaska,17072,NaN,NaN,2616,2315,84,NaN,NaN,7,11,50.7,0.1
1,Alabama,199158,169266.0,29892.0,10009,4074,3026,2818.0,208.0,112,61,29.3,0.3
2,Arkansas,117360,NaN,NaN,7648,3894,2037,NaN,NaN,143,67,36.3,0.7
3,American Samoa,0,NaN,NaN,0,0,0,NaN,NaN,0,0,0.0,0.0
4,Arizona,252768,246468.0,6300.0,10288,3524,6087,5707.0,380.0,169,84,20.5,0.3


In [42]:
# select relevant columns only
covid_stats_df = covid_stats_df[['State/Territory', 'Total Cases', 'Case Rate per 100000', 'Total Deaths', 'Death Rate per 100000']]
covid_stats_df.head()

,State/Territory,Total Cases,Case Rate per 100000,Total Deaths,Death Rate per 100000
0,Alaska,17072,2315,84,11
1,Alabama,199158,4074,3026,61
2,Arkansas,117360,3894,2037,67
3,American Samoa,0,0,0,0
4,Arizona,252768,3524,6087,84


In [43]:
# add a state abbreviation column
covid_stats_df['State'] = state_df['state']
covid_stats_df.head()

,State/Territory,Total Cases,Case Rate per 100000,Total Deaths,Death Rate per 100000,State
0,Alaska,17072,2315,84,11,AK
1,Alabama,199158,4074,3026,61,AL
2,Arkansas,117360,3894,2037,67,AR
3,American Samoa,0,0,0,0,AS
4,Arizona,252768,3524,6087,84,AZ


In [44]:
# rename columns and drop any column with NA
covid_stats_df = covid_stats_df.rename(columns={"state": "State", "Total Cases": "Total_cases", "Case Rate per 100000": "Case_rate_per_100k", "Total Deaths": "Total_deaths", "Death Rate per 100000": "Death_rate_per_100k"})
covid_stats_df = covid_stats_df[["State", "Total_cases", "Case_rate_per_100k", "Total_deaths", "Death_rate_per_100k"]].fillna(0)
covid_stats_df.tail()

,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k
56,WA,112550,1494,2431,32
57,WI,263571,4534,2269,39
58,WV,26547,1470,480,26
59,WY,15409,2667,105,18
60,0,9581770,2895,234264,71


## USA COVID-19 data with latitude and longtitude to create interactive map

In [52]:
# upload file and read to pandas dataframe
file = "../Data/US_COVID-19_stats.csv"
us_covid_df = pd.read_csv(file)
us_covid_df

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-11-06 05:30:35,32.3182,-86.9023,199158,3026,84471.0,111661.0,1.0,4061.808804,1379107.0,NaN,1.519397,84000001,USA,28126.758423,NaN
1,Alaska,US,2020-11-06 05:30:35,61.3707,-152.4044,18174,84,7125.0,10965.0,2.0,2484.331107,781826.0,NaN,0.462199,84000002,USA,106873.261385,NaN
2,American Samoa,US,2020-11-06 05:30:35,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,1768.0,NaN,NaN,16,ASM,3177.512985,NaN
3,Arizona,US,2020-11-06 05:30:35,33.7298,-111.4312,252768,6087,42317.0,204364.0,4.0,3472.699928,1826629.0,NaN,2.408137,84000004,USA,25095.480426,NaN
4,Arkansas,US,2020-11-06 05:30:35,34.9697,-92.3731,117360,2037,104816.0,10507.0,5.0,3888.920553,1392905.0,NaN,1.735685,84000005,USA,46156.244740,NaN
5,California,US,2020-11-06 05:30:35,36.1162,-119.6816,956854,17860,NaN,938994.0,6.0,2421.665822,19266363.0,NaN,1.866533,84000006,USA,48760.513930,NaN
6,Colorado,US,2020-11-06 05:30:35,39.0598,-105.3111,121006,2353,8268.0,110385.0,8.0,2101.259721,2109114.0,NaN,1.944532,84000008,USA,36624.599565,NaN
7,Connecticut,US,2020-11-06 05:30:35,41.5978,-72.7554,77060,4656,9800.0,62604.0,9.0,2161.396824,2433409.0,NaN,6.042045,84000009,USA,68252.822283,NaN
8,Delaware,US,2020-11-06 05:30:35,39.3185,-75.5071,25753,716,13685.0,11352.0,10.0,2644.685981,573791.0,NaN,2.780259,84000010,USA,58925.057817,NaN
9,Diamond Princess,US,2020-11-06 05:30:35,NaN,NaN,49,0,NaN,49.0,88888.0,NaN,NaN,NaN,0.000000,84088888,USA,NaN,NaN


In [46]:
# select relevant columns
us_covid_df = us_covid_df[["Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Mortality_Rate"]]
us_covid_df.head()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Mortality_Rate
0,Alabama,32.3182,-86.9023,199158,3026,84471.0,1.519397
1,Alaska,61.3707,-152.4044,18174,84,7125.0,0.462199
2,American Samoa,-14.2710,-170.1320,0,0,NaN,NaN
3,Arizona,33.7298,-111.4312,252768,6087,42317.0,2.408137
4,Arkansas,34.9697,-92.3731,117360,2037,104816.0,1.735685


In [47]:
# add state column to be filled with state abbreviation
us_covid_df["State"] = "" 
us_covid_df.head()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Mortality_Rate,State
0,Alabama,32.3182,-86.9023,199158,3026,84471.0,1.519397,
1,Alaska,61.3707,-152.4044,18174,84,7125.0,0.462199,
2,American Samoa,-14.2710,-170.1320,0,0,NaN,NaN,
3,Arizona,33.7298,-111.4312,252768,6087,42317.0,2.408137,
4,Arkansas,34.9697,-92.3731,117360,2037,104816.0,1.735685,


In [48]:
# upload States and their abbreviation
us_state_abbrev = {
    
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Diamond Princess': 'DP',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Grand Princess': 'GP',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [61]:
# loop through and assign abbreviation acoordinely
for index, row in us_covid_df.iterrows():
    us_covid_df.loc[index, "State"] = us_state_abbrev[row[0]]

    if row["State"] == "DP":
        us_covid_df.loc[index, "Lat"] = 14.5214
        us_covid_df.loc[index, "Long_"] = 120.9709

    elif row["State"] == "GP":
        us_covid_df.loc[index, "Lat"] = 32.1584
        us_covid_df.loc[index, "Long_"] = 117.5676

us_covid_df

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate,State
0,Alabama,US,2020-11-06 05:30:35,32.3182,-86.9023,199158,3026,84471.0,111661.0,1.0,4061.808804,1379107.0,NaN,1.519397,84000001,USA,28126.758423,NaN,AL
1,Alaska,US,2020-11-06 05:30:35,61.3707,-152.4044,18174,84,7125.0,10965.0,2.0,2484.331107,781826.0,NaN,0.462199,84000002,USA,106873.261385,NaN,AK
2,American Samoa,US,2020-11-06 05:30:35,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,1768.0,NaN,NaN,16,ASM,3177.512985,NaN,AS
3,Arizona,US,2020-11-06 05:30:35,33.7298,-111.4312,252768,6087,42317.0,204364.0,4.0,3472.699928,1826629.0,NaN,2.408137,84000004,USA,25095.480426,NaN,AZ
4,Arkansas,US,2020-11-06 05:30:35,34.9697,-92.3731,117360,2037,104816.0,10507.0,5.0,3888.920553,1392905.0,NaN,1.735685,84000005,USA,46156.244740,NaN,AR
5,California,US,2020-11-06 05:30:35,36.1162,-119.6816,956854,17860,NaN,938994.0,6.0,2421.665822,19266363.0,NaN,1.866533,84000006,USA,48760.513930,NaN,CA
6,Colorado,US,2020-11-06 05:30:35,39.0598,-105.3111,121006,2353,8268.0,110385.0,8.0,2101.259721,2109114.0,NaN,1.944532,84000008,USA,36624.599565,NaN,CO
7,Connecticut,US,2020-11-06 05:30:35,41.5978,-72.7554,77060,4656,9800.0,62604.0,9.0,2161.396824,2433409.0,NaN,6.042045,84000009,USA,68252.822283,NaN,CT
8,Delaware,US,2020-11-06 05:30:35,39.3185,-75.5071,25753,716,13685.0,11352.0,10.0,2644.685981,573791.0,NaN,2.780259,84000010,USA,58925.057817,NaN,DE
9,Diamond Princess,US,2020-11-06 05:30:35,14.5214,120.9709,49,0,NaN,49.0,88888.0,NaN,NaN,NaN,0.000000,84088888,USA,NaN,NaN,DP


In [62]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     if row["State"] == "DP":
#         us_covid_df.loc[index, "Lat"] = 14.5214
#         us_covid_df.loc[index, "Long_"] = 120.9709

#     elif row["State"] == "GP":
#         us_covid_df.loc[index, "Lat"] = 32.1584
#         us_covid_df.loc[index, "Long_"] = 117.5676

    # if row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

# us_covid_df.head()

In [63]:
# rename columns and add recovery percent column
us_covid_df = us_covid_df.rename(columns={"Long_": "Long", "Mortality_Rate":"Death_percent"})
us_covid_df["Recovery_percent"]=us_covid_df.Recovered/us_covid_df.Confirmed
us_covid_df = us_covid_df[["State", "Lat", "Long", "Death_percent", "Recovery_percent"]].fillna(0)
us_covid_df.head()

,State,Lat,Long,Death_percent,Recovery_percent
0,AL,32.3182,-86.9023,1.519397,0.424141
1,AK,61.3707,-152.4044,0.462199,0.392044
2,AS,-14.2710,-170.1320,0.000000,0.000000
3,AZ,33.7298,-111.4312,2.408137,0.167414
4,AR,34.9697,-92.3731,1.735685,0.893115


In [64]:
# merge two df into one dataframe using left joint.
us_covid_combine_df = pd.merge(covid_stats_df, us_covid_df, how='left', on=['State','State'])
us_covid_combine_df.head()

,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k,Lat,Long,Death_percent,Recovery_percent
0,AK,17072,2315,84,11,61.3707,-152.4044,0.462199,0.392044
1,AL,199158,4074,3026,61,32.3182,-86.9023,1.519397,0.424141
2,AR,117360,3894,2037,67,34.9697,-92.3731,1.735685,0.893115
3,AS,0,0,0,0,-14.2710,-170.1320,0.000000,0.000000
4,AZ,252768,3524,6087,84,33.7298,-111.4312,2.408137,0.167414


In [65]:
# save to a csv file
us_covid_combine_df.to_csv('../Data/states.csv')

## World COVID-19 Data by Country

In [66]:
# upload file and read into a pandas dataframe
file = "../Data/World_COVID-19_Stats.csv"
world_covid_df = pd.read_csv(file)
world_covid_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-06 05:24:55,33.93911,67.709953,41935,1554,34440,5941.0,Afghanistan,107.723573,3.705735
1,NaN,NaN,NaN,Albania,2020-11-06 05:24:55,41.15330,20.168300,22721,543,11696,10482.0,Albania,789.526722,2.389860
2,NaN,NaN,NaN,Algeria,2020-11-06 05:24:55,28.03390,1.659600,60169,2011,41244,16914.0,Algeria,137.212244,3.342253
3,NaN,NaN,NaN,Andorra,2020-11-06 05:24:55,42.50630,1.521800,5135,75,3858,1202.0,Andorra,6645.958714,1.460565
4,NaN,NaN,NaN,Angola,2020-11-06 05:24:55,-11.20270,17.873900,12102,299,5350,6453.0,Angola,36.821948,2.470666


In [67]:
# select relevant columns only
world_covid_df = world_covid_df[["Country_Region", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Case-Fatality_Ratio"]]
world_covid_df.head()

,Country_Region,Lat,Long_,Confirmed,Deaths,Recovered,Case-Fatality_Ratio
0,Afghanistan,33.93911,67.709953,41935,1554,34440,3.705735
1,Albania,41.15330,20.168300,22721,543,11696,2.389860
2,Algeria,28.03390,1.659600,60169,2011,41244,3.342253
3,Andorra,42.50630,1.521800,5135,75,3858,1.460565
4,Angola,-11.20270,17.873900,12102,299,5350,2.470666


In [68]:
# rename columns and add a recovery percent column
world_covid_df = world_covid_df.rename(columns={"Long_": "Long", "Case-Fatality_Ratio":"Death_percent", "Country_Region": "Country"})
world_covid_df["Recovery_percent"]=world_covid_df.Recovered/world_covid_df.Confirmed
world_covid_df.head()

,Country,Lat,Long,Confirmed,Deaths,Recovered,Death_percent,Recovery_percent
0,Afghanistan,33.93911,67.709953,41935,1554,34440,3.705735,0.821271
1,Albania,41.15330,20.168300,22721,543,11696,2.389860,0.514766
2,Algeria,28.03390,1.659600,60169,2011,41244,3.342253,0.685469
3,Andorra,42.50630,1.521800,5135,75,3858,1.460565,0.751315
4,Angola,-11.20270,17.873900,12102,299,5350,2.470666,0.442076


In [69]:
# add missing geoinfo to country
for index, row in world_covid_df.iterrows(): 
    if row["Country"] == "Canada":
        world_covid_df.loc[index, "Lat"] = 56.1304
        world_covid_df.loc[index, "Long"] = 106.3468
world_covid_df

,Country,Lat,Long,Confirmed,Deaths,Recovered,Death_percent,Recovery_percent
0,Afghanistan,33.939110,67.709953,41935,1554,34440,3.705735,0.821271
1,Albania,41.153300,20.168300,22721,543,11696,2.389860,0.514766
2,Algeria,28.033900,1.659600,60169,2011,41244,3.342253,0.685469
3,Andorra,42.506300,1.521800,5135,75,3858,1.460565,0.751315
4,Angola,-11.202700,17.873900,12102,299,5350,2.470666,0.442076
...,...,...,...,...,...,...,...,...
3955,West Bank and Gaza,31.952200,35.233200,56672,508,48680,0.896386,0.858978
3956,Western Sahara,24.215500,-12.885800,10,1,8,10.000000,0.800000
3957,Yemen,15.552727,48.516388,2063,601,1375,29.132332,0.666505
3958,Zambia,-13.133897,27.849332,16770,349,15827,2.081097,0.943769


In [70]:
# save file as a csv
world_covid_df.to_csv('../Data/world.csv')

## build SQlite DataBase

In [80]:
# import modules
import sqlite3
from sqlite3 import Error

# series of functions to  build database tables in sqlite database.

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def drop_table(conn, drop_table_sql):
    """ drop a table from the drop_table_sql statement
    :param conn: Connection object
    :param drop_table_sql: a drop TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(drop_table_sql)
    except Error as e:
        print(e)

# build empty tables with correct columns names
def main():
    database = r"../DataBase/covid-19.db"
    sql_create_monthly_table = """CREATE TABLE monthly(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Average_cases float,
                                        Average_death float,
                                        Average_recovery float,
                                        Death_percent float,
                                        Recovery_percent float
                                    ); """

    sql_drop_monthly_table = """DROP TABLE if exists monthly;"""
    
    sql_create_states_table = """CREATE TABLE states(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                                        State text,
                                        Total_cases float,
                                        Case_rate_per_100k float,
                                        Total_deaths float,
                                        Death_rate_per_100k float,
                                        Lat float,
                                        Long float,
                                        Death_percent float,
                                        Recovery_percent float
                                );"""

    sql_drop_states_table = """DROP TABLE if exists states;"""    

    sql_create_world_table = """CREATE TABLE world(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                                        Country text,
                                        Lat float,
                                        Long float,
                                        Confirmed float,
                                        Deaths float,
                                        Recovered float,
                                        Recovery_percent float,
                                        Death_percent float
                                );"""

    sql_drop_world_table = """DROP TABLE if exists world;"""

    # create a database connection
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create covid_monthly table
        drop_table(conn, sql_drop_monthly_table)
        create_table(conn, sql_create_monthly_table)

        # create covid_states table
        drop_table(conn, sql_drop_states_table)
        create_table(conn, sql_create_states_table)

        #create covid_world table
        drop_table(conn, sql_drop_world_table)
        create_table(conn, sql_create_world_table)
        
    # error handling
    else:
        print("Error! cannot create the database connection.")
if __name__ == '__main__':
    main()

In [81]:
# create engine connection to database
engine = create_engine('sqlite:///../DataBase/covid-19.db')

# populate tables with information from dataframes
covid_monthly_df.to_sql(name='monthly', con=engine, if_exists = "append", index=False)
world_covid_df.to_sql(name='world', con=engine, if_exists = "append", index= False)
us_covid_combine_df.to_sql(name='states', con=engine, if_exists = "append", index=False)

In [82]:
# querry monthly data from database. to test and make sure tables are working
pd.read_sql('select * from monthly', engine)

,id,Date,State,Average_cases,Average_death,Average_recovery,Death_percent,Recovery_percent
0,1,2020-01,AK,0.0,0.0,0.0,0.00,0.00
1,2,2020-01,AL,0.0,0.0,0.0,0.00,0.00
2,3,2020-01,AR,0.0,0.0,0.0,0.00,0.00
3,4,2020-01,AS,0.0,0.0,0.0,0.00,0.00
4,5,2020-01,AZ,0.6,0.0,0.6,0.00,100.00
...,...,...,...,...,...,...,...,...
955,956,2021-04,VT,19639.0,227.5,19411.5,1.16,98.84
956,957,2021-04,WA,366438.5,5270.0,361168.5,1.44,98.56
957,958,2021-04,WI,637354.5,7331.0,630023.5,1.15,98.85
958,959,2021-04,WV,142443.0,2685.5,139757.5,1.89,98.11


In [ ]:
# querry states data from database. to test and make sure tables are working
pd.read_sql('select * from states', engine)

In [ ]:
# make an API call to make sure Flask app is working 
import requests
url="http://127.0.0.1:5000/api/v1.0/monthly"
resp=requests.get(url)
resp

In [ ]:
data = resp.json()
data